### Архитектура

+ src:
    + ML_logic.py - реализация основных методов обучения, предсказания, лоад, анлоад и ремув
    + models.py - описания body запроса, с помощью pydantic
+ main.py - здесь хранятся ручки сервера
+ const.py - здесь инициализируется и проверяется на корректность переменная свободных процессов
+ .env - отсюда берутся входные парасетры как из условия задания

In [ ]:
#models.py

from pydantic import BaseModel
from typing import List, Optional


class MLConfig(BaseModel):
    model: str
    task: str
    name: str  # name for file model
    params: Optional[dict]


class BodyFit(BaseModel):
    X: List[List[float]]
    y: List[float]
    config: MLConfig


class BodyPredict(BaseModel):
    X: List[List[float]]
    config: MLConfig

In [ ]:
#ML_logic.py

# Здесь вся мльная составляющая, проверяем тип задачи - регрессия или классификация, и выбираем алгоритм, которые запрашивает
# пользователь в теле запроса
# внутри методов фит и предикт, вызываются методы unload и load соответственно

import os

from src.models import MLConfig
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
import pickle
from dotenv import load_dotenv


load_dotenv()
save_dir = os.getenv('FILENAME')



class MLApps:
    @classmethod
    def fit(cls, X, y, config: MLConfig, process_nums):
        if config.task == "reg":
            if config.model == "Tree":
                model = DecisionTreeRegressor()
            elif config.model == "Boosting":
                model = GradientBoostingRegressor()
            elif config.model == "Linear":
                model = LinearRegression()
        else:
            if config.model == "Tree":
                model = DecisionTreeClassifier()
            elif config.model == "Boosting":
                model = GradientBoostingClassifier()
            elif config.model == "Linear":
                model = LogisticRegression()

        model.fit(X, y)

        filename = save_dir + '/' + config.name + '.sav'

        cls.unload(model, filename) #сохраняем модель в директорию

        with process_nums.get_lock():
            process_nums.value += 1

        return 0

    @classmethod
    def predict(cls, X,  config):
        filename = save_dir + '/' + config.name + '.sav'
        model = cls.load(filename)
        y_predict = model.predict(X)
        return y_predict

    @staticmethod
    def unload(model, filename: str) -> None:
        pickle.dump(model, open(filename, 'wb'))

    @staticmethod
    def load(filename: str):
        try:
            loaded_model = pickle.load(open(filename, 'rb'))
        except (OSError, IOError) as e:
            print("Oops!  That was no valid number.  Try again...")
        return loaded_model

    @classmethod
    def remove(cls, filename) -> None:
        os.remove(filename)

In [ ]:
#main.py

# Здесь описание всех ручек
# выводы необходимых методов и логика печати времени, которым я оперирую в клиентском ноутбуке
# необходимая проверка наличия файла в директории

# многопроцессорность реализована с помощью модуля multiprocessing.Process и вывод клиенту количество свободных процессов.

import os
from fastapi import FastAPI, HTTPException

from src.ML_logic import MLApps, save_dir
from src.models import MLConfig, BodyFit, BodyPredict

import multiprocessing as mp
from datetime import datetime
from const import process_nums
from os.path import getctime

app = FastAPI()


@app.post("/fit")
async def fit_model(body: BodyFit):
    # global process_nums
    if process_nums.value == 0:
        raise HTTPException(status_code=503, detail="нет свободных процессов, попробуйте позже")
    process = mp.Process(target=MLApps.fit, args=(body.X, body.y, body.config, process_nums))
    process.daemon = True
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    process.start()
    with process_nums.get_lock():
        process_nums.value -= 1
    res = {'msg': f'Поставлено на обучение, время - {current_time}, свободных процессов {process_nums.value}, '}
    return res


@app.post("/predict")
async def predict(body: BodyPredict):
    filename = body.config.name + '.sav'
    if filename in os.listdir(save_dir):
        y_pred = MLApps.predict(body.X, body.config)
        time_saved = datetime.fromtimestamp(getctime(save_dir + '/' + filename)).strftime('%H:%M:%S')
        return {'время сохранения модели': {time_saved}, 'predictions': list(y_pred)}
    else:
        raise HTTPException(status_code=400, detail="Нет модели с "
                                                    "таким именем на диске")


@app.post("/remove")
async def remove(body: MLConfig):
    filename = save_dir + '/' + body.name + '.sav'
    MLApps.remove(filename)


@app.get("/remove_all")
async def remove_all():
    dir = save_dir
    for f in os.listdir(dir):
        os.remove(os.path.join(dir, f))


In [ ]:
#const.py

from multiprocessing import Value, cpu_count
import os

cpu_cores = int(os.getenv('NUMBER_OF_CPU'))
cpu_cores_available = cpu_count()
print(type(cpu_cores_available))

if cpu_cores <= cpu_cores_available - 1:
    process_nums = Value('i', cpu_cores)  # оставляем один процессор под сервер
else:
    process_nums = Value('i', cpu_cores_available - 1)


In [ ]:
# .env

FILENAME='output'
NUMBER_OF_CPU=4
NUMBER_OF_MODELS=1